## Получение данных Тинькофф Пульс

In [78]:
"""Sync Client for TCS PULSE api"""
from typing import Dict, Optional

import httpx
from fake_useragent import UserAgent
from fake_useragent.errors import FakeUserAgentError

from tpulse import settings

try:
    ua = UserAgent().random
except (IndexError, FakeUserAgentError):
    ua = settings.USER_AGENT


class ClientBase:
    """Base class for API client"""

    def __init__(self, base_url: str):
        headers = {
            "Content-type": "application/json",
            "Accept": "application/json",
            "User-agent": ua,
        }
        data = {"appName": "invest", "origin": "web", "platform": "web"}
        self._client = httpx.Client(base_url=base_url, headers=headers, params=data)

    def __enter__(self) -> "ClientBase":
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    def close(self):
        """Close network connections"""
        self._client.close()

    def _get(self, url, data, timeout=settings.TIMEOUT_SEC):
        """GET request to Dadata API"""
        response = self._client.get(url, params=data, timeout=timeout)
        response.raise_for_status()
        return response.json()


class UserClient(ClientBase):
    """User class for tpulse api"""

    BASE_URL = "https://www.tinkoff.ru/api/invest-gw/social/v1/"

    def __init__(self):
        super().__init__(base_url=self.BASE_URL)

    def user_info(self, name: str) -> Optional[Dict]:
        """get user info by username"""
        url = "profile/nickname/%s" % name
        response = self._get(url, data=None)
        return response["payload"] if response["status"] == "Ok" else None

    def user_posts(self, user_id: str, cursor: int, **kwargs) -> Optional[Dict]:
        """get user posts by user id"""
        url = "profile/%s/post" % user_id
        data = {"limit": 30, "cursor": cursor}
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None

    def user_operations(self, user_id: str, cursor: int, **kwargs) -> Optional[Dict]:
        """get user oprations by user id"""
        url = "profile/%s/instrument" % user_id
        data = {"limit": 30, "cursor": cursor}
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None

    def user_yield(self, user_id: str, cursor: int, **kwargs) -> Optional[Dict]:
        """get user yield (year/month) by user id"""
        url = "profile/%s/portfolio/yield" % user_id
        data = {"limit": 30}
        if cursor:
            data["cursor"] = cursor
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None

    def portfolio_structure(self, user_id: str, cursor: int, **kwargs) -> Optional[Dict]:
        """get structure of user portfolio by user id"""
        url = "profile/%s/portfolio/analytics" % user_id
        data = {"limit": 30}
        if cursor:
            data["cursor"] = cursor
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None

    def user_ticker_operations(
        self,
        user_id: str,
        ticker: str,
        class_code: str,
        cursor: int,
        **kwargs
    ) -> Optional[Dict]:
        """get operations by user and ticker id"""
        url = f"profile/{user_id}/operation/instrument/{ticker}/{class_code}"
        data = {"limit": 30}
        if cursor:
            data["cursor"] = cursor
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None


class PostClient(ClientBase):
    """Ticker class for tpulse api"""

    BASE_URL = "https://www.tinkoff.ru/api/invest-gw/social/v1/"

    def __init__(self):
        super().__init__(base_url=self.BASE_URL)

    def posts(self, ticker: str, cursor: int, **kwargs) -> Optional[Dict]:
        """get post info by ticker"""
        url = "post/instrument/%s" % ticker
        data = {"limit": 30, "cursor": cursor}
        data.update(kwargs)
        response = self._get(url, data)
        return response["payload"] if response["status"] == "Ok" else None


class PulseClient:
    """Sync client for tpulse api"""

    def __init__(self):
        self._user = UserClient()
        self._post = PostClient()

    def test(self):
        """test func"""
        pass

    def get_user_info(self, name: str) -> Optional[Dict]:
        """Get user info"""
        return self._user.user_info(name)

    def get_posts_by_user_id(
        self, user_id: str, cursor: int = 999999999, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 posts for user"""
        return self._user.user_posts(user_id, cursor, **kwargs)

    def get_posts_by_ticker(
        self, ticker: str, cursor: int = 999999999, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 posts for ticker"""
        return self._post.posts(ticker, cursor, **kwargs)

    def get_operations_by_user_id(
        self, user_id: str, cursor: int = 999999999, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 operations for user"""
        return self._user.user_operations(user_id, cursor, **kwargs)

    def get_operations_by_user_and_ticker(
        self, user_id: str, ticker: str, class_code: str, cursor: int = 0, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 operations for user"""
        return self._user.user_ticker_operations(user_id, ticker, class_code, cursor, **kwargs)

    def get_yield_by_user_id(
        self, user_id: str, cursor: int = 0, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 operations for user"""
        return self._user.user_yield(user_id, cursor, **kwargs)

    def get_portfolio_structure_by_user_id(
        self, user_id: str, cursor: int = 0, **kwargs
    ) -> Optional[Dict]:
        """Collect last 30 operations for user"""
        return self._user.portfolio_structure(user_id, cursor, **kwargs)

    def __enter__(self) -> "PulseClient":
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()

    def close(self):
        """Close network connections"""
        self._user.close()
        self._post.close()


In [112]:
# информация о доходности, портфеле и сделках пользователей доступна только после авторизации
session_id = "XXX"

In [86]:
from pprint import pp

pulse = PulseClient()

user_nickname = "Alex_Abramyan"

### Информация о пользователе

#### Структура ответа

```json
{"id": "48000d0d-f0e0-425a-b466-f94f3b3bb864",
 "type": "personal",
 "nickname": "Alex_Abramyan",
 "status": "open",
 "image": "4470504e-7411-476b-b618-c73e183b3a91",
 "block": False,
 "description": "🧠 Канал для умных https://t.me/thefinexpert \n"
                "\n"
                "Личка в Телеграмм @alex_abramyan",
 "followersCount": 44494,
 "followingCount": 82,
 "isLead": False,
 "isMuted": False,
 "serviceTags": [{"id": "popular"}, {"id": "advice"}],
 "statistics": {"totalAmountRange": {"lower": 1000000, "upper": 3000000},
                "yearRelativeYield": 42.04,
                "monthOperationsCount": 65},
 "popularHashtags": [{"id": "a97342cb-b993-46a8-aa7d-29c8ab1ce643", "title": "интересное"},
                     {"id": "1e646571-e69c-407f-8a54-85f0739244da", "title": "пульс_оцени"}],
 "donationActive": True,
 "isVisible": True,
 "baseTariffCategory": "unauthorized",
 "strategies": [{"id": "a67a5905-8a50-47a4-bd06-cd602617cef9",
                 "title": "Акции Гонконга",
                 "riskProfile": "aggressive",
                 "relativeYield": -14.18,
                 "baseCurrency": "hkd",
                 "score": 4,
                 "portfolioValues": [],
                 "characteristics": [{"id": "recommended-base-money-position-quantity",
                                      "value": "19\xa0000 HK$",
                                      "subtitle": "советуем вложить"},
                                     {"id": "minimum-base-money-position-quantity",
                                      "value": "20\xa0000 HK$",
                                      "subtitle": "Минимальная сумма"},
                                     {"id": "slaves-count",
                                      "value": "6",
                                      "subtitle": "подписаны"},
                                     {"id": "expected-relative-yield",
                                      "value": "30% в год",
                                      "subtitle": "Прогноз"},
                                     {"id": "relative-yield",
                                      "value": "−14,18%",
                                      "subtitle": "за все время"},
                                     {"id": "short-description",
                                      "value": "Инвестируем в китайские акции с умом",
                                      "subtitle": "Короткое описание"},
                                     {"id": "owner-description",
                                      "value": "Инвестирует с 2018 г., ACCA, квал. инвестор",
                                      "subtitle": "Описание владельца"},
                                     {"id": "master-portfolio-top-positions",
                                      "value": "US62914V1061.png,US88032Q1094.png,KYG960071028.png",
                                      "subtitle": "Топ торгуемых бумаг"}],
                 "tags": [],
                 "type": "public"}],
 "investId": "9213d597-ef54-4319-b89c-00deb4a760cc"}
 ```

#### Запрос

In [103]:
user_info = pulse.get_user_info(user_nickname)

### Отзывы пользователя

#### Структура ответа

```json
[{"id": "092f11b6-0fe9-431a-a093-8a9ac69d602c",
  "text": "",
  "likesCount": 47,
  "commentsCount": 3,
  "isLiked": False,
  "inserted": "2023-10-20T17:22:17.291+03:00",
  "isEditable": False,
  "instruments": [{"type": "share",
                   "ticker": "OZON",
                   "lastPrice": 2761.0,
                   "currency": "rub",
                   "image": "ISSUANCEOZON.png",
                   "briefName": "Ozon Holdings PLC",
                   "relativeDailyYield": 0.15,
                   "price": 2750.5,
                   "relativeYield": 0.38}],
  "profiles": [],
  "serviceTags": [],
  "profileId": "48000d0d-f0e0-425a-b466-f94f3b3bb864",
  "nickname": "Alex_Abramyan",
  "image": "4470504e-7411-476b-b618-c73e183b3a91",
  "postImages": [],
  "hashtags": [],
  "owner": {"id": "48000d0d-f0e0-425a-b466-f94f3b3bb864",
            "nickname": "Alex_Abramyan",
            "image": "4470504e-7411-476b-b618-c73e183b3a91",
            "donationActive": False,
            "block": False,
            "serviceTags": [{"id": "popular"}, {"id": "advice"}]},
  "reactions": {"totalCount": 47,
                "counters": [{"type": "like", "count": 43},
                             {"type": "rocket", "count": 3},
                             {"type": "buy-up", "count": 1}]},
  "content": {"type": "simple",
              "text": "❗$OZON уходит с NASDAQ! Что это значит? \n"
                      " \n"
                      "Сегодня {$OZON} объявил о добровольном делистинге своих "
                      "АДР с NASDAQ. Некоторых такие новости напугали, но на "
                      "самом деле шаг логичный и абсолютно нормальный. "
                      "Разбираемся \n"
                      " \n"
                      "🛑 Напомню, в США торги АДР ОЗОН не идут уже более "
                      "полутора лет. Но при этом компания несет расходы по "
                      "предоставлению отчетности и вынуждена соблюдать "
                      "требования американских регуляторов. Ранее ОЗОН "
                      "предпринял две попытки оспорить решение о делистинге, "
                      "но получил отказ",
              "instruments": [{"type": "share",
                               "ticker": "OZON",
                               "lastPrice": 2761.0,
                               "currency": "rub",
                               "image": "ISSUANCEOZON.png",
                               "briefName": "Ozon Holdings PLC",
                               "relativeDailyYield": 0.15,
                               "price": 2750.5,
                               "relativeYield": 0.38}],
              "hashtags": [],
              "profiles": [],
              "images": [],
              "strategies": []},
  "baseTariffCategory": "unauthorized",
  "isBookmarked": False,
  "status": "published"},
  ...
]
```

#### Запрос

In [104]:
# все отзывы пользователя без ограничения по времени
user_id = user_info["id"]
user_posts = []
next_cursor = 999999999
while next_cursor:
    response = pulse.get_posts_by_user_id(user_id, cursor=next_cursor)
    user_posts.extend(response.get("items", []))
    next_cursor = response.get("nextCursor", 0)

### Отзывы по тикеру

#### Структура ответа

```json
[{"id": "e3a312fa-6d3a-4919-ac37-6e1db0c99f10",
  "text": "",
  "likesCount": 6,
  "commentsCount": 0,
  "isLiked": False,
  "inserted": "2023-10-23T10:35:30.085+03:00",
  "isEditable": False,
  "instruments": [{"type": "share",
                   "ticker": "AAPL",
                   "lastPrice": 171.84,
                   "currency": "usd",
                   "image": "US0378331005.png",
                   "briefName": "Apple",
                   "relativeDailyYield": -0.99,
                   "price": 172.89,
                   "relativeYield": -0.61},
                  {"type": "share",
                   "ticker": "CHMF",
                   "lastPrice": 1398.0,
                   "currency": "rub",
                   "image": "RU0009046510.png",
                   "briefName": "Северсталь",
                   "relativeDailyYield": -0.84,
                   "price": 1401.0,
                   "relativeYield": -0.21},
                  {"type": "share",
                   "ticker": "ROSN",
                   "lastPrice": 600.9,
                   "currency": "rub",
                   "image": "RU000A0J2Q06.png",
                   "briefName": "Роснефть",
                   "relativeDailyYield": -0.33,
                   "price": 603.0,
                   "relativeYield": -0.35},
                  {"type": "share",
                   "ticker": "GMKN",
                   "lastPrice": 16970.0,
                   "currency": "rub",
                   "image": "nornikel.png",
                   "briefName": "Норильский никель",
                   "relativeDailyYield": -0.89,
                   "price": 17118.0,
                   "relativeYield": -0.86}],
  "profiles": [],
  "serviceTags": [],
  "profileId": "c9f9ca9e-771a-4b8a-a76d-d2242b079ba5",
  "nickname": "Redhead83",
  "image": "5b8c76f6-3ccb-48d6-b20e-4238aeca3f48",
  "postImages": [{"id": "da01791c-265f-4b74-9ca8-d5691fc3f439",
                  "height": 1200,
                  "width": 1200}],
  "hashtags": [{"id": "8284581f-fe17-4109-baca-e4626593d636",
                "title": "хочу_в_дайджест"}],
  "owner": {"id": "c9f9ca9e-771a-4b8a-a76d-d2242b079ba5",
            "nickname": "Redhead83",
            "image": "5b8c76f6-3ccb-48d6-b20e-4238aeca3f48",
            "donationActive": False,
            "block": False,
            "serviceTags": []},
  "reactions": {"totalCount": 6,
                "counters": [{"type": "like", "count": 3},
                             {"type": "buy-up", "count": 1},
                             {"type": "get-rid", "count": 1},
                             {"type": "rocket", "count": 1}]},
  "content": {"type": "simple",
              "text": "🌱 Пятничный рост российского рынка акций возглавили "
                      "экспортёры: {$ROSN} Роснефть (+1,3%), {$CHMF} "
                      "Северсталь (+1%) и {$GMKN} Норникель (+2%). Металлурги "
                      "сегодня поделятся операционными результатами за III "
                      "квартал: возможно, инвесторы ждут хороших отчётов.\xa0\n"
                      "\n"
                      "🐉 В Китае сегодня утром распродажи тоже усилились. "
                      "Инвесторов напугало расследование против Foxconn, "
                      "важнейшего поставщика Apple {$AAPL} и одного из "
                      "крупнейших работодателей Китая.\n",
              "instruments": [{"type": "share",
                               "ticker": "AAPL",
                               "lastPrice": 171.84,
                               "currency": "usd",
                               "image": "US0378331005.png",
                               "briefName": "Apple",
                               "relativeDailyYield": -0.99,
                               "price": 172.89,
                               "relativeYield": -0.61},
                              {"type": "share",
                               "ticker": "CHMF",
                               "lastPrice": 1398.0,
                               "currency": "rub",
                               "image": "RU0009046510.png",
                               "briefName": "Северсталь",
                               "relativeDailyYield": -0.84,
                               "price": 1401.0,
                               "relativeYield": -0.21},
                              {"type": "share",
                               "ticker": "ROSN",
                               "lastPrice": 600.9,
                               "currency": "rub",
                               "image": "RU000A0J2Q06.png",
                               "briefName": "Роснефть",
                               "relativeDailyYield": -0.33,
                               "price": 603.0,
                               "relativeYield": -0.35},
                              {"type": "share",
                               "ticker": "GMKN",
                               "lastPrice": 16970.0,
                               "currency": "rub",
                               "image": "nornikel.png",
                               "briefName": "Норильский никель",
                               "relativeDailyYield": -0.89,
                               "price": 17118.0,
                               "relativeYield": -0.86}],
              "hashtags": [{"id": "8284581f-fe17-4109-baca-e4626593d636",
                            "title": "хочу_в_дайджест"}],
              "profiles": [],
              "images": [{"id": "da01791c-265f-4b74-9ca8-d5691fc3f439",
                          "height": 1200,
                          "width": 1200}],
              "strategies": []},
  "baseTariffCategory": "unauthorized",
  "isBookmarked": False,
  "status": "published"},
...
]
```

### Запрос

In [106]:
ticker = "AAPL"
ticker_posts = pulse.get_posts_by_ticker(ticker)

### Сделки пользователя

#### Структура ответа

```json
{
    "trackingId": "iuptF1gFJJ",
    "time": "2023-10-20T15:05:04.781+03:00",
    "status": "Ok",
    "payload": {
        "hasPrev": false,
        "nextCursor": 2351174147,
        "hasNext": true,
        "items": [
            {
                "type": "etf",
                "ticker": "TRUR",
                "classCode": "TQTF",
                "showName": "Тинькофф Вечный портфель RUB",
                "currency": "rub",
                "image": "TRUR.png",
                "statistics": {
                    "totalOperationsCount": 1284,
                    "maxTradeDateTime": "2023-10-20T13:35:31.822+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "TRNFP",
                "classCode": "TQBR",
                "showName": "Транснефть - привилегированные акции",
                "currency": "rub",
                "image": "RU0009091573.png",
                "statistics": {
                    "totalOperationsCount": 3,
                    "maxTradeDateTime": "2023-09-07T13:28:28.246+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "GLTR",
                "classCode": "TQBR",
                "showName": "Globaltrans Investment PLC",
                "currency": "rub",
                "image": "US37949E2046.png",
                "statistics": {
                    "totalOperationsCount": 4,
                    "maxTradeDateTime": "2023-09-05T12:13:30.167+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "OZON",
                "classCode": "TQBR",
                "showName": "Ozon Holdings PLC",
                "currency": "rub",
                "image": "ISSUANCEOZON.png",
                "statistics": {
                    "totalOperationsCount": 15,
                    "maxTradeDateTime": "2023-09-04T10:18:10.789+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "LKOH",
                "classCode": "TQBR",
                "showName": "ЛУКОЙЛ",
                "currency": "rub",
                "image": "RU0009024277.png",
                "statistics": {
                    "totalOperationsCount": 34,
                    "maxTradeDateTime": "2023-09-04T10:13:23.870+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "SMLT",
                "classCode": "TQBR",
                "showName": "ГК Самолет",
                "currency": "rub",
                "image": "RU000A102RX6.png",
                "statistics": {
                    "totalOperationsCount": 15,
                    "maxTradeDateTime": "2023-09-04T10:09:00.288+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "HHR",
                "classCode": "SPBEQRU",
                "showName": "HeadHunter Group PLC",
                "currency": "usd",
                "image": "US42207L1061.png",
                "statistics": {
                    "totalOperationsCount": 5,
                    "maxTradeDateTime": "2023-08-30T11:03:17.277+03:00"
                }
            },
            ...
            {
                "type": "share",
                "ticker": "2015",
                "classCode": "SPBHKEX",
                "showName": "Li Auto",
                "currency": "hkd",
                "image": "US50202M1027.png",
                "statistics": {
                    "totalOperationsCount": 2,
                    "maxTradeDateTime": "2023-07-14T10:48:30.099+03:00"
                }
            },
            {
                "type": "share",
                "ticker": "PLZL",
                "classCode": "TQBR",
                "showName": "Полюс",
                "currency": "rub",
                "image": "PolyusGold.png",
                "statistics": {
                    "totalOperationsCount": 19,
                    "maxTradeDateTime": "2023-07-10T16:46:20.909+03:00"
                }
            }
        ],
        "statistics": {
            "monthOperationsCount": 69
        }
    }
}
```

#### Запрос

In [113]:
user_operations = pulse.get_operations_by_user_id(user_id, sessionId=session_id)

### Сделки по тикеру

#### Структура ответа

```json
{
    "trackingId": "aViVYC7iHJ",
    "time": "2023-10-20T15:07:52.088+03:00",
    "status": "Ok",
    "payload": {
        "hasNext": false,
        "items": [
            {
                "tradeDateTime": "2023-09-07T13:28:28.246+03:00",
                "action": "sell",
                "averagePrice": 138350,
                "relativeYield": -2.4300
            },
            {
                "tradeDateTime": "2023-09-05T12:15:59.227+03:00",
                "action": "sell",
                "averagePrice": 139750,
                "relativeYield": -1.4500
            },
            {
                "tradeDateTime": "2023-09-04T10:06:05.950+03:00",
                "action": "buy",
                "averagePrice": 141800
            }
        ],
        "statistics": {
            "totalOperationsCount": 1,
            "maxTradeDateTime": "2023-10-20T15:07:52.088+03:00"
        }
    }
}
```

#### Запрос

In [115]:
ticker = "TRUR"
class_code = None
for operation in user_operations["items"]:
    if operation["ticker"] == ticker:
        class_code = operation["classCode"]
        break
user_ticker_operations = pulse.get_operations_by_user_and_ticker(
    user_id,
    ticker,
    class_code,
    cursor = 0,
    sessionId=session_id
)

### Состав портфеля

#### Структура ответа

```json
{
    "payload": {
        "totalAmountRange": {
            "lower": 1000000,
            "upper": 3000000
        },
        "types": [
            {"title": "Акции", "percent": 94.16},
            {"title": "Фонды", "percent": 5.57},
            {"title": "Валюта и Металлы", "percent": 0.27}
        ],
        "issuers": [
            {"title": "Ростелеком", "percent": 40.04},
            {"title": "ТМК", "percent": 32.56},
            {"title": "Globaltrans Investment", "percent": 7.97},
            {"title": "OZON", "percent": 5.1},
            {"title": "HeadHunter Group", "percent": 4.34},
            {"title": "Тинькофф Капитал", "percent": 4.11},
            ...
        ],
        "sectors": [
            {"title": "Телекоммуникации", "percent": 41.12},
            {"title": "Сырьевая промышленность", "percent": 32.56},
            {"title": "Потребительские товары", "percent": 11.87},
            {"title": "Машиностроение и транспорт", "percent": 7.97},
            {"title": "Другое", "percent": 5.57},
            {"title": "Энергетика", "percent": 0.65},
            {"title": "Валюта и Металлы", "percent": 0.27}
        ],
        "currencies": [
            {"title": "RUB", "percent": 91.24},
            {"title": "HKD", "percent": 4.42},
            {"title": "USD", "percent": 4.34},
            {"title": "EUR", "percent": 0}
        ]
    }
}
```

#### Запрос

In [116]:
user_yield = pulse.get_portfolio_structure_by_user_id(id, sessionId=session_id)

### Доходность по годам/месяцам

#### Структура ответа

```json
{
    "payload": {
        "years": [
            {"cut": "2023-01-01", "relativeYield": 26.68},
            {"cut": "2022-01-01", "relativeYield": -57.14},
            {"cut": "2021-01-01", "relativeYield": 34.59},
            {"cut": "2020-01-01", "relativeYield": 27.70},
            {"cut": "2019-01-01", "relativeYield": 1.13}
        ],
        "months": [
            {"cut": "2023-10-01", "relativeYield": 1.21},
            {"cut": "2023-09-01", "relativeYield": -10.93},
            {"cut": "2023-08-01", "relativeYield": -1.80},
            ...
            {"cut": "2020-03-01", "relativeYield": -9.17},
            {"cut": "2020-02-01", "relativeYield": -14.13},
            {"cut": "2020-01-01", "relativeYield": 5.72}
        ]
    }
}
```

#### Запрос

In [117]:
user_yield = pulse.get_yield_by_user_id(id, sessionId=session_id)